# A simpl(er) Introduction to Hierarchical Models 
### Naive Bayesians, 2021


#### Hierarachical Multivariate Regression
* Hierarchical Univariate Regression
* Hierarchical Multivariate Regression

## TOC

1. Recap
2. Univariate Model
3. Multivariate model - basically show the math behind it


#### Notebook Setup

In [1]:
%load_ext nb_black
%reload_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML

display(HTML("<style>.container { width:85% !important; }</style>"))

<IPython.core.display.Javascript object>

In [2]:
import os
from scipy import stats
import pandas as pd
import numpy as np
from typing import Dict

# ML libraries
import pymc3 as pm
from sklearn.linear_model import LinearRegression

# Plotting and viz
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["font.size"] = 15

<IPython.core.display.Javascript object>

#### Univariate Regression - Radon Gas Example

Suppose the radon concentration for a specific house, $i$, in county, $c$ is related to if the house has a basement with the relationship:  

$$
\begin{align*}
\text{Pooled:  } \text{log_radon}_{i, c} = {} & \alpha + \beta \hspace{1mm} \text{floor}_{i, c} + \epsilon_{i, c}, \\
\text{Unpooled:  }  \text{log_radon}_{i, c} = {} & \alpha_{c} + \beta_{c} \hspace{1mm} \text{floor}_{i, c} + \epsilon_{i, c}
\end{align*}
$$

where $\alpha_{c}$ is the average radon concentration, $\text{floor}_{i, c}  = 0$ if the house has a basement  $\text{floor}_{i, c}  = 1$ if the house doesn't have a basement. 

In [3]:
# Load data
def load_radon_data() -> pd.DataFrame:
    data = pd.read_csv(pm.get_data("radon.csv"))
    data = data[["county", "floor", "log_radon"]]
    return data


radon_data = load_radon_data()

counties = radon_data["county"].drop_duplicates().to_list()

print(f"Num counties: {len(counties)}")

# Map each county with an index
county_index_mapping = {county: index for index, county in enumerate(counties)}

radon_data = radon_data.assign(
    **{"county_index": lambda x: x["county"].map(county_index_mapping)}
)
display(radon_data)

Num counties: 85


,county,floor,log_radon,county_index
0,AITKIN,1.0,0.832909,0
1,AITKIN,0.0,0.832909,0
2,AITKIN,0.0,1.098612,0
3,AITKIN,0.0,0.095310,0
4,ANOKA,0.0,1.163151,1
...,...,...,...,...
914,WRIGHT,0.0,1.871802,83
915,WRIGHT,0.0,1.526056,83
916,WRIGHT,0.0,1.629241,83
917,YELLOW MEDICINE,0.0,1.335001,84


<IPython.core.display.Javascript object>

In [ ]:
# Build individual models
with pm.Model() as ind_radon_model:
    # Prior
    # ---> Regression Coefficients
    alpha = pm.Normal("alpha", mu=0, sigma=10, shape=1)
    beta = pm.Normal("beta", mu=0, sigma=10, shape=1)
    # ---> Noise
    eps = pm.HalfCauchy("eps", 5)

    # Likelihood
    mu = alpha + beta * radon_data["floor"]
    y = pm.Normal("obs", mu=mu, sigma=eps, observed=radon_data["log_radon"])

    # Posterior
    trace = pm.sample(draws=2000)

In [ ]:
pm.plot_posterior(trace)
plt.show()

In [ ]:
with ind_radon_model:
    display(pm.summary(trace))

In [4]:
# Build individual models
with pm.Model() as unpooled_radon_model:
    # Prior
    # ---> Regression Coefficients
    alpha = pm.Normal("alpha", mu=0, sigma=10, shape=len(counties))
    beta = pm.Normal("beta", mu=0, sigma=10, shape=len(counties))
    # ---> Noise
    eps = pm.HalfCauchy("eps", 5)

    # Likelihood
    mu = (
        alpha[radon_data["county_index"]]
        + beta[radon_data["county_index"]] * radon_data["floor"]
    )
    y = pm.Normal("obs", mu=mu, sigma=eps, observed=radon_data["log_radon"])

    # Posterior
    trace = pm.sample(draws=2000)

/opt/anaconda3/envs/homework/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
/opt/anaconda3/envs/homework/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [eps, beta, alpha]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 109 seconds.
/opt/anaconda3/envs/homework/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])


<IPython.core.display.Javascript object>

In [6]:
with unpooled_radon_model:
    display(pm.summary(trace))

/opt/anaconda3/envs/homework/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
alpha[0],0.684,0.414,-0.091,1.456,0.005,0.004,7009.0,5077.0,7007.0,3015.0,1.0
alpha[1],0.954,0.103,0.754,1.140,0.001,0.001,7805.0,7605.0,7809.0,2752.0,1.0
alpha[2],1.462,0.700,0.147,2.724,0.008,0.007,6961.0,5692.0,6941.0,3088.0,1.0
alpha[3],1.711,0.410,0.965,2.481,0.005,0.003,7315.0,7043.0,7352.0,3026.0,1.0
alpha[4],1.335,0.413,0.562,2.110,0.005,0.004,6832.0,5196.0,6978.0,3264.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
beta[81],-0.039,9.974,-17.492,19.694,0.109,0.208,8317.0,1147.0,8316.0,2630.0,1.0
beta[82],-1.971,0.467,-2.884,-1.129,0.006,0.004,6981.0,6757.0,6994.0,3458.0,1.0
beta[83],-0.790,0.752,-2.250,0.538,0.009,0.009,7531.0,3541.0,7549.0,3225.0,1.0
beta[84],-0.090,9.954,-18.948,18.270,0.101,0.194,9627.0,1316.0,9618.0,2414.0,1.0


<IPython.core.display.Javascript object>

In [7]:
# Build individual models
with pm.Model() as hierarchical_radon_model:
    # Prior
    # -> Global prior
    mu_alpha_glob = pm.Normal("mu_alpha", mu=0, sigma=10)
    sigma_alpha_glob = pm.HalfCauchy("sig_alpha", 5)

    mu_beta_glob = pm.Normal("mu_beta", mu=0, sigma=10)
    sigma_beta_glob = pm.HalfCauchy("sig_beta", 5)

    # ---> Regression Coefficients
    alpha = pm.Normal(
        "alpha", mu=mu_alpha_glob, sigma=sigma_alpha_glob, shape=len(counties)
    )
    beta = pm.Normal(
        "beta", mu=mu_beta_glob, sigma=sigma_beta_glob, shape=len(counties)
    )
    # ---> Noise
    eps = pm.HalfCauchy("eps", 5)

    # Likelihood
    mu = (
        alpha[radon_data["county_index"]]
        + beta[radon_data["county_index"]] * radon_data["floor"]
    )
    y = pm.Normal("obs", mu=mu, sigma=eps, observed=radon_data["log_radon"])

    # Posterior
    trace = pm.sample(draws=2000)

/opt/anaconda3/envs/homework/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
/opt/anaconda3/envs/homework/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [eps, beta, alpha, sig_beta, mu_beta, sig_alpha, mu_alpha]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 144 seconds.
There were 8 divergences after tuning. Increase `target_accept` or reparameterize.
There were 46 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


<IPython.core.display.Javascript object>

In [8]:
with hierarchical_radon_model:
    display(pm.summary(trace))

/opt/anaconda3/envs/homework/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
mu_alpha,1.491,0.053,1.394,1.589,0.002,0.001,821.0,807.0,837.0,1007.0,1.00
mu_beta,-0.644,0.082,-0.800,-0.492,0.003,0.002,562.0,562.0,565.0,768.0,1.01
alpha[0],1.210,0.247,0.753,1.665,0.004,0.003,3330.0,3330.0,3357.0,2947.0,1.00
alpha[1],0.990,0.095,0.820,1.170,0.002,0.001,3597.0,3519.0,3610.0,2376.0,1.00
alpha[2],1.509,0.262,1.016,1.986,0.005,0.003,2859.0,2859.0,2852.0,2764.0,1.00
...,...,...,...,...,...,...,...,...,...,...,...
beta[83],-0.655,0.281,-1.223,-0.147,0.005,0.004,3648.0,2172.0,3299.0,1942.0,1.02
beta[84],-0.639,0.308,-1.255,-0.077,0.005,0.004,3624.0,2642.0,3242.0,1691.0,1.01
sig_alpha,0.326,0.046,0.241,0.411,0.001,0.001,1030.0,1030.0,1028.0,2124.0,1.00
sig_beta,0.273,0.115,0.040,0.459,0.016,0.011,52.0,52.0,41.0,15.0,1.05


<IPython.core.display.Javascript object>

In [9]:
pm.plot_posterior(trace)
plt.show()

/opt/anaconda3/envs/homework/lib/python3.7/site-packages/arviz/data/io_pymc3.py:92: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,
/opt/anaconda3/envs/homework/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/opt/anaconda3/envs/homework/lib/python3.7/site-packages/arviz/plots/plot_utils.py:496: UserWarning: rcParams['plot.max_subplots'] (40) is smaller than the number of variables to plot (175) in plot_posterior, generating only 40 plots
  UserWarning,


OSError: [Errno 86] Bad CPU type in executable: 'convert'

<IPython.core.display.Javascript object>


\begin{align*}
y_{a, i} =  \pmb{w}_{a}^\textsf{T}\pmb{x}_{a, i} + \epsilon_{i}  \;\;\;\;
y_{b, i} = \pmb{w}_{b}^\textsf{T}\pmb{x}_{b, i} + \epsilon_{i} \\
\end{align*}





\begin{align*}
\pmb{y}_{a} = {} & \begin{bmatrix}y_{a, 1} \\ \vdots \\  y_{a, N_{a}}  \end{bmatrix}  \;\;\;\;
\pmb{y}_{b} =  \begin{bmatrix}y_{b, 1} \\ \vdots \\  y_{b, N_{b}}  \end{bmatrix} \\
\end{align*}



\begin{align*}
\pmb{y} = {} & \begin{bmatrix} \pmb{y}_{a} \\   \pmb{y}_{b} \end{bmatrix} \;\;\;\;
\end{align*}



\begin{align*}
\pmb{y}_{a} = {} & \pmb{X}_{a}\pmb{w}_{a} + \pmb{\epsilon} \;\;\;\;
\pmb{y}_{b} =   \pmb{X}_{b}\pmb{w}_{b} + \pmb{\epsilon}  \\
\end{align*}



\begin{align*}
\pmb{X}_{a}= {} & \begin{bmatrix} \pmb{x}^\textsf{T}_{a, 1} \\ \vdots \\  \pmb{x}^\textsf{T}_{a, N_{a}}  \end{bmatrix} \;\;\;\;
\pmb{X}_{b}=  \begin{bmatrix} \pmb{x}^\textsf{T}_{b, 1} \\ \vdots \\  \pmb{x}^\textsf{T}_{b, N_{b}}  \end{bmatrix} \\
\end{align*}


\begin{align*}
\pmb{X} = {} & \begin{bmatrix} \pmb{X}_{a} \\   \pmb{X}_{b} \end{bmatrix} \;\;\;\;
\end{align*}


In [ ]:
n = 1000
w_opt_a = np.array([1, 3.5, 6])
w_opt_b = np.array([-1, -3.5, -6])
m = len(w_opt)

X = stats.norm.rvs(size=(n * 2, m), loc=0, scale=1)

X_a, X_b = X[0:n], X[n:]


y = np.append((X_a @ w_opt_a), (X_b @ w_opt_b)) + stats.norm.rvs(
    size=n * 2, loc=0, scale=0.5
)

### Pooled Model

The pooled model considers both the data set (Least squares solution)


$$
\begin{align*}
\hat{\pmb{w}}_{\textsf{pooled}} = \left(\pmb{X}^\textsf{T} \pmb{X} \right)^{-1} \pmb{X}^\textsf{T}\pmb{y}
\end{align*}
$$


In [ ]:
w = np.linalg.inv((X.T @ X)) @ (X.T @ y)

print("Direct computation", w)

lin_mod = LinearRegression(fit_intercept=False)
lin_mod.fit(X, y)
print("Sklearn Linear Regession", lin_mod.coef_)

In [ ]:
with pm.Model() as mod:
    w_ = pm.Normal(name="w", mu=0, sigma=10, shape=m)
    sigma = pm.InverseGamma("sigma", mu=1, sigma=10)
    obs = pm.Normal(name="obs", observed=y, mu=X @ w_, sigma=sigma)
    trace = pm.sample(draws=1000)

In [ ]:
pm.plot_posterior(trace)
plt.show()

Let 
\begin{align*}
\hat{\pmb{w}} =  \begin{bmatrix}  \hat{\pmb{w}}_a  ,   \hat{\pmb{w}}_b\end{bmatrix} 
\end{align*}




\begin{align*}
\pmb{X} = {} & \begin{bmatrix} \pmb{X}_{a} \\   \pmb{X}_{b} \end{bmatrix} \;\;\;\;
\end{align*}


\begin{align*}
\tilde{\pmb{y}} = {} & \pmb{X} \hat{\pmb{w}}   \\
 = {} & \begin{bmatrix} \pmb{X}_{a} \\   \pmb{X}_{b} \end{bmatrix} \begin{bmatrix}  \hat{\pmb{w}}_a  ,   \hat{\pmb{w}}_b\end{bmatrix} \\
  = {} & \begin{bmatrix} \hat{\pmb{y}}_{a}, \ \tilde{\pmb{y}}_{ab} \\  \tilde{\pmb{y}}_{ba}, \hat{\pmb{y}}_{b} \end{bmatrix} 
\end{align*}


\begin{align*}
\hat{\pmb{y}} = {} & \begin{bmatrix} \hat{\pmb{y}}_{a} \\ \hat{\pmb{y}}_{b} \end{bmatrix} 
\end{align*}




\begin{align*}
\hat{\pmb{y}} = {} & \tilde{\pmb{y}}\textsf{.iloc}\left( \begin{bmatrix} 0 & 0 \\ 1 & 0 \\ 2 & 0 \\ N_a & 0 \\ N_a + 1 & 1 \\ N_a + 2 & 1 \\ N_a + 3 & 1 \\ N_a + N_b & 1 \end{bmatrix}  \right)
\end{align*}



In [ ]:
index_cols = np.append(
    np.zeros(len(X_a), dtype=np.int8), np.ones(len(X_b), dtype=np.int8)
)
index_rows = np.arange(n * 2)

with pm.Model() as ind_mod:
    w_ind = pm.Normal(name="w", mu=0, sigma=10, shape=(m, 2))
    sigma = pm.InverseGamma("sigma", mu=1, sigma=10)
    #  mu = pm.Deterministic(name="mu", var=(X @ w_ind)[index_rows, index_cols])
    mu = (X @ w_ind)[index_rows, index_cols]
    obs = pm.Normal(name="obs", observed=y, mu=mu, sigma=sigma)
    trace = pm.sample(draws=1500)

In [ ]:
trace.get_values(varname="mu").shape

In [ ]:
trace.get_values(varname="w").shape

In [ ]:
pm.plot_posterior(trace, var_names=["w"])
plt.show()

In [ ]:
with ind_mod:
    display(pm.summary(trace))

In [ ]:
w_both = np.array([w_opt_a, w_opt_b]).T
w_both

(X @ w_both)[index_rows, index_cols]